# Sample Creation

## Setup

In [ ]:
# this step reloads machine in order to prevent any error
%load_ext autoreload
%autoreload 2

In [7]:
# import statements
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import Pipette
import time

import numpy as np
import pandas as pd
import os

In [ ]:
# defines the jubilee to call
jubilee = Jub.Machine(address='192.168.1.2', simulated = False) 

In [ ]:
# load deck
deck = jubilee.load_deck('lab_automation_deck.json')

In [29]:
# this loads the well plate for the destination for samples (sample_holder_name, position)
samples = jubilee.load_labware('48_vial_plate v2.stl', 2)
# this line of code defines the boundary conditions of the well plate requires 3 points
upper_left = (25.7,179.9)
upper_right = (124.9, 179.9)
lower_right = (124.9, 116.4)
samples.manual_offset([upper_left,upper_right,lower_right])
samples.load_manualOffset()

NameError: name 'jubilee' is not defined

In [30]:
# plan on having a samples_trash for an alloquat plate to keep a constant concentration
samples_trash = jubilee.load_labware('48_vial_plate v2.stl',5) #used for the aloquates
upper_left = (25.7,179.9)
upper_right = (124.9, 179.9)
lower_right = (124.9, 116.4)
samples_trash.manual_offset([upper_left,upper_right,lower_right])
samples_trash.load_manualOffset()

NameError: name 'jubilee' is not defined

In [ ]:
# this is same as lines above just for stock solution
stocks = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 3)
upper_left = (167.4,176.8)
upper_right = (258.0, 176.8)
lower_right = (258.0, 119.8)
stocks.manual_offset([upper_left,upper_right,lower_right])
stocks.load_manualOffset()

In [ ]:
# same as above just for pipette tip rack
tiprack = jubilee.load_labware('opentrons_96_tiprack_300ul.json', 0)
upper_left = (27.0,82.7)
upper_right = (125.6,82.7)
lower_right = (125.8,19.4)
tiprack.manual_offset([upper_left,upper_right,lower_right])
tiprack.load_manualOffset()

In [ ]:
# defines position of trash can for tips
trash = jubilee.load_labware('agilent_1_reservoir_290ml.json', 1)

In [ ]:
deck.safe_z

In [ ]:
# this code block defines all of our stocks
water_stock = stocks[0].bottom(+5) 
blue1_stock = stocks[1].bottom(+5) #stock A: 0.10 M
blue2_stock = stocks[2].bottom(+5) #         0.15 M
blue3_stock = stocks[3].bottom(+5) #         0.20 M
red1_stock = stocks[4].bottom(+5)  #stock B: 0.10 M
red2_stock = stocks[5].bottom(+5)  #         0.15 M
red3_stock = stocks[6].bottom(+5)  #         0.20 M
# include however many we need, remember to include however many stocks there are
# this list will be a reference lise that will be used to return the index value of which stock to choose 
reference_stocks_blue = [0, 0.1, 0.15, 0.2]
reference_stocks_red = [0,0,0,0, 0.1, 0.15, 0.2]

## Load Tools

In [ ]:
P300 = Pipette.Pipette.from_config(3,'Pipette','P300_config.json')
jubilee.load_tool(P300)

P300.add_tiprack(tiprack)
P300.trash = trash[0]

## Experiment Functions

In [25]:
def transfer_stock(v, stock, target_sample):
    '''
    This function will take a volume to add, and from which stock to use

    Input - v (volume)(float)
          - stock (int) -position of the desired stock
          - sample (int) -position of the current sample
    Output - void, purpose of function    
    '''
    P300.transfer(v, source_well=stocks[stock].bottom(+5) ,
                  destination_well = samples[target_sample].bottom(+4), 
                  blowout=True, 
                  new_tip='once' )

In [24]:
def transfer_mix(v, stock, sample):
    '''
    This function will take a volume to add, and from which stock to use

    Input - v (volume)(float)
          - stock (int) index location of stock
          - sample (int) sample index location
 
    Output - void, purpose of function    
    '''
    #P300.transfer(v, source_well=stocks[stock],
                  #destination_well=samples[sample], 
                  #blowout=True, 
                  #mix_after=(4,300),
                  #new_tip='always')
    #these are for the later alloquate
    #P300.aspirate(1000, samples[sample])
    #P300.dispense(1000, samples_trash[sample])
    P300.pickup_tip()
    P300.aspirate(v, stocks[stock].bottom(+5))
    P300.dispense(v, samples[sample].bottom(+4))
    P300.blowout()

    # manual mixing step
    for i in range(3):
        P300.aspirate(300,samples[sample].bottom(+2))
        P300.dispense(300,samples[sample].bottom(+2))

    # move to allequat
    P300.aspirate(300, samples[sample].bottom(+2))
    P300.dispense(300, samples_trash[sample].bottom(+2))
    P300.blowout()
    P300.drop_tip()

In [ ]:
def transfer_water(v,sample):
    '''
    This function will take a volume to add, and from which stock to use

    Input - v (volume)(float)
          - sample (int)
    Output - void, purpose of function    
    '''
    # source well is always the same
    P300.transfer(v, source_well=stocks[0].bottom(+5),
                  destination_well = samples[sample].bottom(+4), 
                  blowout=True, 
                  new_tip='once' )

In [ ]:
def mixer(sample_location_index): # hold off on this function for now
    '''
    this function will mix a chosen sample, it is done this way in order to reuse the same tip for mixing and not using the stock solution tip
    Input - sample_location(int) is an index location of which sample to mix
    Output - none
    '''
    P300.transfer(300, samples[sample_location_index].bottom(+3),
                  sample_trash[sample_location_index].bottom(+3),
                  blowout=True,
                  mix_before=(3,300),
                  new_tip='always' )

# Data for experiment #

In [8]:
os.getcwd()

'/Users/Igi/science-jubilee/critical_material_project'

In [9]:
data = pd.read_csv('./data/for_igi.2.0.csv')

In [22]:
# this is first time step, all the others can be done by increasing the index
# the dt1 is a list of [vol_a, vol_b, vol_h20, stock_a, stock_b]
#dt1 = data.loc[0]
data.head()
reference_stocks_blue = [0, 0.1, 0.2, 0.3]
reference_stocks_red = [0, 0, 0, 0, 0.10, 0.15, 0.20] # first zeros 

In [28]:
vol_a = data['vol_A']
vol_b = data['vol_B']
vol_water = data['vol_water']
stock_a = data['stock_A']
stock_b = data['stock_B']
a_index_list = []
b_index_list = []

vol_a = vol_a * 1000/3.3
vol_b = vol_b * 1000/3.3
vol_water = vol_water * 1000/3.3

for i in range(len(stock_a)):
    temp = reference_stocks_blue.index(stock_a[i])
    a_index_list.append(temp)

for i in range(len(stock_b)):
    temp = reference_stocks_red.index(stock_b[i])
    b_index_list.append(temp)

print(a_index_list, b_index_list)
print(vol_a, vol_b, vol_water)

[0, 1, 1, 1, 1, 1, 1, 2, 2, 2] [5, 4, 4, 4, 4, 4, 4, 4, 4, 4]
0      0.000000
1      8.151515
2     24.303030
3     55.090909
4    112.363636
5    187.363636
6    260.272727
7    161.848485
8    189.333333
9    213.454545
Name: vol_A, dtype: float64 0    202.030303
1    248.818182
2    121.303030
3     47.212121
4     12.454545
5      2.848485
6      1.424242
7      1.000000
8      0.696970
9      0.484848
Name: vol_B, dtype: float64 0    101.000000
1     46.060606
2    157.424242
3    200.727273
4    178.212121
5    112.818182
6     41.333333
7    140.151515
8    113.000000
9     89.090909
Name: vol_water, dtype: float64


In [ ]:
jubilee.pickup_tool(P300)

In [ ]:
# this one is for stock a
transfer_water(vol_b[0],0)

In [ ]:
# stock b
transfer_stock(vol_b[0],1,0)

In [ ]:
# water
transfer_mix(vol_a[0],0,0)

In [ ]:
# mix test
mixer(0)

In [ ]:
# for loop for a single sample. Adding in multiple sample files will take more
for i in range(len(vol_a)):
    if vol_water[i] > 0:
        transfer_water(vol_water[i],0)
    if vol_a[i] > 0:
        transfer_stock(vol_a[i],a_index_list[i],0)
    if vol_b[i] > 0:
        transfer_mix(vol_b[i],b_index_list[i],0)
    if vol_b[i] == 0:
        mixer(0)